In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 14.9 MB/s 
     |████████████████████████████████| 163 kB 45.8 MB/s 
     |████████████████████████████████| 7.6 MB 55.3 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from transformers import  BertTokenizer
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, SequentialSampler
from transformers import BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
import torch.optim
import numpy as np
import pandas as pd
import time
import datetime
import random
from sklearn.metrics import confusion_matrix

In [ ]:
path_biased='/content/drive/MyDrive/Hackathones/BBVA Hakathon 2022/Bases de datos/CMS Dataset/Subset proccessed/clean_biased_data.csv'
path_notbiased='/content/drive/MyDrive/Hackathones/BBVA Hakathon 2022/Bases de datos/CMS Dataset/Subset proccessed/clean_notbiased_data.csv'

In [ ]:
!pip install transformers
!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/pytorch_weights.tar.gz 
!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/vocab.txt 
!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/config.json 
!tar -xzvf pytorch_weights.tar.gz
!mv config.json pytorch/.
!mv vocab.txt pytorch/.

--2022-10-21 13:49:07--  https://users.dcc.uchile.cl/~jperez/beto/cased_2M/pytorch_weights.tar.gz
Resolving users.dcc.uchile.cl (users.dcc.uchile.cl)... 192.80.24.4, 200.9.99.211
Connecting to users.dcc.uchile.cl (users.dcc.uchile.cl)|192.80.24.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 409871727 (391M) [application/x-gzip]
Saving to: ‘pytorch_weights.tar.gz’

pytorch_weights.tar 100%[===================>] 390.88M  8.21MB/s    in 63s     

2022-10-21 13:50:11 (6.25 MB/s) - ‘pytorch_weights.tar.gz’ saved [409871727/409871727]

--2022-10-21 13:50:11--  https://users.dcc.uchile.cl/~jperez/beto/cased_2M/vocab.txt
Resolving users.dcc.uchile.cl (users.dcc.uchile.cl)... 200.9.99.211, 192.80.24.4
Connecting to users.dcc.uchile.cl (users.dcc.uchile.cl)|200.9.99.211|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 242120 (236K) [text/plain]
Saving to: ‘vocab.txt’

vocab.txt           100%[===================>] 236.45K   285KB/s    in 0.8s

In [ ]:
biased=pd.read_csv(path_biased)
not_biased=pd.read_csv(path_notbiased)
# Juntar datasets
data_final=pd.concat([biased, not_biased])


In [ ]:
data_final['label'] = np.where(data_final['sexist']== True, 1, 0)
df=data_final[['spanish_text','label']]


In [ ]:
df=df.dropna()

In [ ]:
run_on = 'cuda'
device = torch.device(run_on)

# Load the dataset into a pandas dataframe.

reviews = df['spanish_text']
sentiment = df['label']

# Split dataset
X_train, X_val, y_train, y_val = train_test_split(reviews, 
sentiment, stratify=sentiment, test_size=0.2, random_state=42)

# Report datasets lenghts
print('Training set length : {}'.format(len(X_train)))
print('Validation set length : {}'.format(len(X_val)))

Training set length : 2877
Validation set length : 720


In [ ]:
tokenizer = BertTokenizer.from_pretrained("pytorch/",
            do_lower_case=True)


In [ ]:
def preprocessing(dataset):
    input_ids = []
    attention_mask = []
    for doc in dataset:
        encoded_doc = tokenizer.encode_plus(doc,
                   add_special_tokens=True, max_length=80,
                   truncation=True,pad_to_max_length=True)
        input_ids.append(encoded_doc['input_ids'])
        attention_mask.append(encoded_doc['attention_mask'])
    return (torch.tensor(input_ids),
           torch.tensor(attention_mask))

In [ ]:
X_train_inputs, X_train_masks = preprocessing(X_train)
X_val_inputs, X_val_masks = preprocessing(X_val)

# Report max n° tokens in a sentence
max_len = max([torch.sum(sen) for sen in X_train_masks])
print('Max n°tokens in a sentence: {0}'.format(max_len))

Max n°tokens in a sentence: 46


In [ ]:
# Data loaders
batch_size = 32


y_val_labels = torch.tensor(y_val.values)
y_train_labels = torch.tensor(y_train.values)


def dataloader(x_inputs, x_masks, y_labels):
    data = TensorDataset(x_inputs, x_masks, y_labels)
    sampler = SequentialSampler(data)
    dataloader = DataLoader(data, sampler=sampler,
                 batch_size=batch_size,
                 num_workers=0)
    return dataloader

train_dataloader = dataloader(X_train_inputs, X_train_masks,
                   y_train_labels)
val_dataloader = dataloader(X_val_inputs, X_val_masks, 
                 y_val_labels)

In [ ]:
def set_seed(value):
    random.seed(value)
    np.random.seed(value)
    torch.manual_seed(value)
    torch.cuda.manual_seed_all(value)
set_seed(42)

# Create model and optimizer
model = BertForSequenceClassification.from_pretrained(
        "pytorch/", num_labels=2, output_attentions=False,
         output_hidden_states=False)

optimizer = AdamW(model.parameters(),
                  lr = 4e-5,
                  eps = 1e-6
                  )

if run_on == 'cuda':
    model.cuda()

# Define number of epochs
epochs = 5

total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
            num_warmup_steps = 0, 
            num_training_steps = total_steps)

Some weights of the model checkpoint at pytorch/ were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pytorch/

In [ ]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

#function to compute accuracy
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
#function to train the model
def training(n_epochs, training_dataloader,
             validation_dataloader):
    # ========================================
    #               Training 
    # ========================================
    print('======= Training =======')
    for epoch_i in range(0,n_epochs):
        # Perform one full pass over the training set
        print("")
        print('======= Epoch {:} / {:} ======='.format(
             epoch_i + 1, epochs))
        # Measure how long the training epoch takes.
        t0 = time.time()
        # Reset the total loss for this epoch.
        total_loss = 0
        # Put the model into training mode.
        model.train()
        # For each batch of training data
        for step, batch in enumerate(training_dataloader):
            batch_loss = 0
            # Unpack this training batch from dataloader
            #   [0]: input ids, [1]: attention masks, 
            #   [2]: labels
            b_input_ids,b_input_mask, b_labels = tuple(
                                t.to(device) for t in batch)

            # Clear any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass 
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask,
                            labels=b_labels)

            # pull loss value out of the output tuple
            loss = outputs[0]
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass 
            loss.backward()

            # Clip the norm of the gradients to 1.0.
            torch.nn.utils.clip_grad_norm_(model.parameters(),
                                            1.0)

            # Update parameters
            # ¿take a step using the computed gradient
            optimizer.step()
            scheduler.step()

            print('batch loss: {0} | avg loss: {1}'.format(
                  batch_loss, total_loss/(step+1)))
        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(train_dataloader)


        print("")
        print("  Average training loss: {0:.2f}".
             format(avg_train_loss))
        print("  Training epoch took: {:}".format(
              format_time(time.time() - t0)))

        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, 
        # measure accuracy on the validation set.

        print("")
        print("======= Validation =======")

        t0 = time.time()

        # Put the model in evaluation mode
        model.eval()

        # Tracking variables
        eval_loss, eval_accuracy = 0, 0
        all_logits = []
        all_labels = []
        # Evaluate data for one epoch
        for step, batch in enumerate(validation_dataloader):
            # Add batch to device
            # Unpack this training batch from our dataloader.
            #   [0]: input ids, [1]: attention masks,
            #   [2]: labels
            b_input_ids, b_input_mask, b_labels = tuple(
                                t.to(device) for t in batch)


            # Model will not to compute gradients
            with torch.no_grad():
                # Forward pass 
                # This will return the logits 
                outputs = model(b_input_ids,
                                token_type_ids=None,
                                attention_mask=b_input_mask)

            # The "logits" are the output values 
            # prior to applying an activation function 
            logits = outputs[0]

            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            b_labels = b_labels.to('cpu').numpy()

            # Save batch logits and labels 
            # We will use thoses in the confusion matrix
            predict_labels = np.argmax(
                             logits, axis=1).flatten()
            all_logits.extend(predict_labels.tolist())
            all_labels.extend(b_labels.tolist())

            # Calculate the accuracy for this batch
            tmp_eval_accuracy = flat_accuracy(
                                logits, b_labels)
            # Accumulate the total accuracy.
            eval_accuracy += tmp_eval_accuracy

        # Report the final accuracy for this validation run.
        print("  Accuracy: {0:.2f}".
              format(eval_accuracy / (step+1)))
        print("  Validation took: {:}".format(
             format_time(time.time() - t0)))

    #print the confusion matrix"
    conf = confusion_matrix(
           all_labels, all_logits, normalize='true')
    print(conf)
    print("")
    print("Training complete")




In [ ]:
#call function to train the model
training(epochs, train_dataloader, val_dataloader)

======= Training =======

======= Epoch 1 / 5 =======
batch loss: 0.6691396236419678 | avg loss: 0.6691396236419678
batch loss: 0.6766266226768494 | avg loss: 0.6728831231594086
batch loss: 0.6426947116851807 | avg loss: 0.6628203193346659
batch loss: 0.645072877407074 | avg loss: 0.6583834588527679
batch loss: 0.667499840259552 | avg loss: 0.6602067351341248
batch loss: 0.6369637250900269 | avg loss: 0.6563329001267751
batch loss: 0.5520808100700378 | avg loss: 0.6414397444043841
batch loss: 0.6164883375167847 | avg loss: 0.6383208185434341
batch loss: 0.45601338148117065 | avg loss: 0.6180644366476271
batch loss: 0.5569202899932861 | avg loss: 0.611950021982193
batch loss: 0.5177004337310791 | avg loss: 0.6033818775957281
batch loss: 0.43776416778564453 | avg loss: 0.5895804017782211
batch loss: 0.46439608931541443 | avg loss: 0.5799508392810822
batch loss: 0.44313156604766846 | avg loss: 0.570178034050124
batch loss: 0.3964209258556366 | avg loss: 0.5585942268371582
batch loss: 0.32

In [ ]:
import torch
torch.save(model, '/content/drive/MyDrive/Hackathones/BBVA Hakathon 2022/BETO model/Bias Classification Beto Model/model')



In [ ]:
saved_model = torch.load('/content/drive/MyDrive/Hackathones/BBVA Hakathon 2022/BETO model/Bias Classification Beto Model/model')

In [ ]:
saved_model.eval()

In [ ]:
text = 'Las hombres no lloran'
tokens = tokenizer.tokenize(text)

In [ ]:
tokens

['las', 'hombres', 'no', 'llora', '##n']

In [ ]:



encoded_doc = tokenizer.encode_plus(text,
    add_special_tokens=True, max_length=80,
truncation=True,pad_to_max_length=True)







/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
encoded_doc

{'input_ids': [4, 1089, 2567, 1084, 27446, 30935, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [ ]:
input_ids_1=[]
attention_mask_1=[]

input_ids_1.append(encoded_doc['input_ids'])
attention_mask_1.append(encoded_doc['attention_mask'])

In [ ]:
run_on = 'cuda'
device = torch.device(run_on)
saved_model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
#test_ids = torch.cat(torch.tensor(input_ids_1))
#test_attention_mask = torch.cat(torch.tensor(attention_mask_1))

# Forward pass, calculate logit predictions
with torch.no_grad():
  output = saved_model(torch.tensor(input_ids_1).to(device), token_type_ids = None, attention_mask = torch.tensor(attention_mask_1).to(device))

#outputs = model(b_input_ids,
#                            token_type_ids=None,
#                            attention_mask=b_input_mask,
#                            labels=b_labels)





prediction = 'Sezgado' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'No sezgo'

print('Input Sentence: ', text)
print(output.logits.cpu().numpy())
print('Predicted Class: ', prediction)


Input Sentence:  Las hombres no lloran
[[-3.1662169  2.9503167]]
Predicted Class:  Sezgado


In [ ]:
list(data_final[data_final['sexist']==True]['spanish_text'])

['todos mis hijos han crecido con juegos de computadora pero no estoy interesado los veo como una cosa de hombres',
 'las mujeres tienen mas intuicion que los hombres',
 'las mujeres no pertenecen a la politica',
 'en promedio los hombres son mas arrogantes que las mujeres',
 'una mujer nunca se sentira verdaderamente realizada en la vida si no tiene una relacion comprometida a largo plazo con un hombre',
 'mmmm buena suerte cuentanos como te va todavia creo que tu y deberian entrar en mas cocina menos quejas mmmm',
 'odio tener chicos como maestros',
 'una mujer no puede decirme nada sobre deportes lo siento creci en una casa loca por los deportes',
 'las mujeres no pueden conducir la unica excepcion danica patrick',
 'no soy sexista pero las habilidades y destrezas de los hombres estan muy por encima de las de las mujeres mucho mas entretenido olimpicos',
 'no soy sexista pero simplemente no creo en que las mujeres sean presidentas',
 'si una mujer no sabe cocinar y cuidar la casa no

In [ ]:
1